## 构建CNN完成Kaggle的MNIST

In [1]:
import pandas as pd
# 用于分层抽样创建验证集
from sklearn.model_selection import StratifiedShuffleSplit
from tensorflow import keras
# 预处理数据集
import tensorflow as tf
import numpy as np

In [2]:
# 用于训练的总数据集data
data = pd.read_csv('train.csv')

# 测试集
test = pd.read_csv('test.csv')

In [3]:
# 使用StratifiedShuffleSplit类进行分层抽样
split = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)

# 按label属性进行划分并生成训练集和验证集
for train_index, valid_index in split.split(data, data['label']):
    train = data.loc[train_index]
    valid = data.loc[valid_index]

# 划分样本与标签
train_X = train.drop(['label'], axis=1)
train_Y = train['label']

# 验证集也划分一下
valid_X = valid.drop(['label'], axis=1)
valid_Y = valid['label']

In [4]:
# 预处理数据集使其维度匹配
train_X = tf.reshape(train_X, [-1, 28, 28, 1])
valid_X = tf.reshape(valid_X, [-1, 28, 28, 1])

test = tf.reshape(test, [-1, 28, 28, 1])

2022-02-12 17:46:01.356335: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-12 17:46:01.360122: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-12 17:46:01.360555: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-12 17:46:01.361397: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [5]:
# 构建模型
model = keras.models.Sequential([
    keras.layers.Conv2D(64, 7, activation='relu', padding='same',
                        input_shape=[28, 28, 1]),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
    keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    keras.layers.Conv2D(256, 3, activation='relu', padding='same'),
    keras.layers.MaxPooling2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
])

In [6]:
# 编译模型
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    metrics=['accuracy']
)

In [7]:
history = model.fit(train_X, train_Y, epochs=30,
                    validation_data=(valid_X, valid_Y))

Epoch 1/30


2022-02-12 17:46:02.727166: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8204
2022-02-12 17:46:03.739025: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1182/1182 [==============================] - 9s 6ms/step - loss: 1.0950 - accuracy: 0.6353 - val_loss: 0.1242 - val_accuracy: 0.9676
Epoch 2/30
1182/1182 [==============================] - 9s 7ms/step - loss: 0.3637 - accuracy: 0.8867 - val_loss: 0.0806 - val_accuracy: 0.9802
Epoch 3/30
1182/1182 [==============================] - 8s 7ms/step - loss: 0.2204 - accuracy: 0.9365 - val_loss: 0.0736 - val_accuracy: 0.9833
Epoch 4/30
1182/1182 [==============================] - 9s 8ms/step - loss: 0.1555 - accuracy: 0.9554 - val_loss: 0.0647 - val_accuracy: 0.9860
Epoch 5/30
1182/1182 [==============================] - 9s 7ms/step - loss: 0.1261 - accuracy: 0.9657 - val_loss: 0.0645 - val_accuracy: 0.9874
Epoch 6/30
1182/1182 [==============================] - 8s 7ms/step - loss: 0.1000 - accuracy: 0.9731 - val_loss: 0.0568 - val_accuracy: 0.9890
Epoch 7/30
1182/1182 [==============================] - 8s 7ms/step - loss: 0.0829 - accuracy: 0.9763 - val_loss: 0.0615 - val_accuracy: 0.9879
Epo

In [8]:
# 进行预测
results = model.predict(test)

# 转换格式
results = np.argmax(results, axis=1)
results = pd.Series(results, name='Label')

# 生成预测文件
submission = pd.concat([pd.Series(range(1, 28001), name="ImageId"), results],
                       axis=1)
submission.to_csv('submit_cnn2.csv', index=False)

In [9]:
# 保存模型
model.save('cnn2.h5')